In [21]:
#!/usr/bin/env python

from random import choice
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry.avro import AvroSerializer
from confluent_kafka.schema_registry import SchemaRegistryClient



config = {
    'bootstrap.servers': 'pkc-921jm.us-east-2.aws.confluent.cloud:9092',
    'sasl.username':     'JGHMRINQ3BUAN42H',
    'sasl.password':     'sOqraI6DVErzvSd3AV2t+TsprixcHW4qv42FRJw8c9j4+vsiUqfII5Xr+YlR2zzG',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms':   'PLAIN',
    'acks':              'all'
}

# Create Producer instance
producer = Producer(config)
topic = 'postgre_sink_test'


scr_client = SchemaRegistryClient({'url': 'https://psrc-57o65q.us-west2.gcp.confluent.cloud',
                            'basic.auth.user.info': f"PCPLW5LBTZIBC464:ixJNPatfkL1wH0+RXhLMZ3MVJ2nhmE1yf5YiIkoz//ab1RPcgh0TueDgHK/Qjc+l"
                            })


In [17]:
def convert_to_dict(message,ctx):
    return dict(message)

In [26]:
schema_str = scr_client.get_schema(100002).schema_str
avro_serializer = AvroSerializer(scr_client,
                                     schema_str,
                                     convert_to_dict)

string_serializer = StringSerializer('utf_8')


In [7]:

def delivery_callback(err, msg):
    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))

In [29]:
import json
from uuid import uuid4
from faker import Faker
import random

def generate_fake_data():
    fake = Faker()
    data = {
        'user_id': str(uuid4()),
        'name': fake.name(),
        'age': random.randint(18,45),
        'team': fake.job(),
    }
    return data


In [45]:

from uuid import uuid4

for i in range(10):
    message  = generate_fake_data()
    producer.produce(topic=topic,
                                key=string_serializer(str(uuid4())),
                                value=avro_serializer(message, SerializationContext(topic, MessageField.VALUE)),
                                on_delivery=delivery_callback)
    producer.poll(10000)
    producer.flush()


User record b'0c46b6a2-e0e3-46bb-9ceb-554ace47b9cc' successfully produced to postgre_sink_test [2] at offset 5
User record b'c25412bf-6c3c-477d-9a34-f2453e89737e' successfully produced to postgre_sink_test [3] at offset 5
User record b'4209ef18-ca87-4b63-b9ff-857b74562540' successfully produced to postgre_sink_test [4] at offset 7
User record b'bf3b6c0f-ae71-4197-a98c-14f3e8b62fea' successfully produced to postgre_sink_test [4] at offset 8
User record b'3797ecd8-51bf-4e08-bc74-7b9195024500' successfully produced to postgre_sink_test [3] at offset 6
User record b'c46a07f1-0161-4fc4-9a59-646898f4c425' successfully produced to postgre_sink_test [3] at offset 7
User record b'0403cbe3-9d54-4e0f-aeb7-4e88a9981247' successfully produced to postgre_sink_test [4] at offset 9
User record b'a0f90743-1bd0-4e7c-b865-40899ca275d3' successfully produced to postgre_sink_test [2] at offset 6
User record b'29a4e6b9-8ad7-4730-bbb2-1d050586c4dd' successfully produced to postgre_sink_test [5] at offset 9
U

In [43]:
message = {'user_id': '3fe4a16b-40ad-4894-9e5b-5ae51b3c71e7',
 'name': 'Erin Salazar',
 'age': 32,
 'team': 'Medical illustrator'}
producer.produce(topic=topic,
                                key=string_serializer(str(uuid4())),
                                value=avro_serializer(message, SerializationContext(topic, MessageField.VALUE)),
                                on_delivery=delivery_callback)

In [41]:
message

{'user_id': '17b7ae44-e8d8-480d-96c8-c92a59bf111d',
 'name': 'Monique Hardin',
 'age': 20,
 'team': 'Music tutor'}